In [1]:
import win32com.client
import configparser
from pywinauto import application
from pywinauto import timings
import pyautogui
import os
import time

In [6]:
class cp_login:
    def __init__(self):
        self.obj_CpUtil_CpCybos = win32com.client.Dispatch('CpUtil.CpCybos')
        self.config = configparser.ConfigParser()

    def kill_client(self):
        os.system('taskkill /IM coStarter* /F /T')
        os.system('taskkill /IM CpStart* /F /T')
        os.system('taskkill /IM DibServer* /F /T')
        os.system('wmic process where "name like \'%coStarter%\'" call terminate')
        os.system('wmic process where "name like \'%CpStart%\'" call terminate')
        os.system('wmic process where "name like \'%DibServer%\'" call terminate')

    # 일반 로그인
    def login(self, prj='cp'):
        self.config.read('C:\\DAISHIN\\CYBOS.ini')
        id = self.config.get('cybos', 'id')
        pwd = self.config.get('cybos', 'password')
        cert = self.config.get('cybos', 'cert')
        
        if not self.connected():
            #print('Cybos Plus - 이미 연결된 클라이언트가 있음.')
            self.disconnect()
            #self.kill_client()
            #print('Cybos Plus - 기존 연결 종료')
            app = application.Application()
            app.start('C:\\DAISHIN\\STARTER\\ncStarter.exe /prj:{} /id:{} /pwd:{} /pwdcert:{} /autostart'.format(prj,id,pwd,cert))
            print('Cybos Plus - 로그인 중...')
            print('Cybos Plus - 접속 중...')
        else:
            print('Cybos Plus - 이미 연결되어 있음.')
            return True
        while not self.connected():
            #time.sleep(1)
            pass
        print('Cybos Plus - 접속 성공.')
        return True
    
    # 모의투자 로그인
    def login_virtual(self, prj='cp'):
        self.config.read('C:\\DAISHIN\\CYBOS.ini')
        id = self.config.get('cybos virtual', 'id')
        pwd = self.config.get('cybos virtual', 'password')
        cert = self.config.get('cybos virtual', 'cert')
        
        if not self.connected():
            self.disconnect()
            app = application.Application()
            app.start('C:\\DAISHIN\\STARTER\\ncStarter.exe /prj:{0}'.format(prj))
            dlg = timings.wait_until_passes(20, 0.5, lambda: app.connect(title='CYBOS Starter')).Dialog
            btn_virtual = dlg.모의투자
            btn_virtual.click()

            # pass edit
            id_edit = dlg.Edit
            id_edit.set_focus()
            id_edit.type_keys(id)

            # cert edit
            cert_edit = dlg.Edit3
            cert_edit.set_focus()
            cert_edit.type_keys(cert)

            # pass edit
            pass_edit = dlg.Edit2
            pass_edit.set_focus()
            pass_edit.type_keys(pwd)

            # login
           
            btn = dlg.연결Button
            btn.click()

            # 약간 대기하고 상시 모의투자 참가 클릭
            time.sleep(30)
            pyautogui.press('enter')

            # 공지사항 닫기
            while(True):
                try:
                    app = application.Application().connect(title_re='공지사항')
                    print('공지사항 창 닫힘.')
                    break
                except:
                    pass

    def connected(self):
        b_connected = self.obj_CpUtil_CpCybos.IsConnect
        if b_connected == 0:
            return False
        return True

    def disconnect(self):
        if self.connected():
            self.obj_CpUtil_CpCybos.PlusDisconnect()

In [7]:
if __name__ == '__main__':
    login = cp_login()
    login.login()

Cybos Plus - 로그인 중...
Cybos Plus - 접속 중...
Cybos Plus - 접속 성공.
